In [ ]:
import pandas as pd

In [ ]:
ed_path = '/home/jun/data/MIMIC/mimic-iv-ed/1.0/ed/'

ed_filename_dict = {'diagnosis':'diagnosis.csv', 'edstays':'edstays.csv',  'medrecon':'medrecon.csv',  'pyxis':'pyxis.csv',  'triage':'triage.csv',  'vitalsign':'vitalsign.csv'}

mimic_iv_path = '/home/jun/data/MIMIC/mimiciv/1.0/'
mimic_iv_core = mimic_iv_path + 'core/'
mimic_iv_hosp = mimic_iv_path + 'hosp/'
mimic_iv_icu = mimic_iv_path + 'icu/'

icu_filename_dict = {"chartevents":"chartevents.csv","datetimeevents":"datetimeevents.csv","d_items":"d_items.csv","icustays":"icustays.csv","inputevents":"inputevents.csv","outputevents":"outputevents.csv","procedureevents":"procedureevents.csv"}
core_filename_dict = {"patients":"patients.csv", "admissions":"admissions.csv", "transfers":"transfers.csv"}
hosp_filename_dict = {"d_hcpcs":"d_hcpcs.csv","d_icd_diagnoses":"d_icd_diagnoses.csv","d_labitems":"d_labitems.csv","emar":"emar.csv","hcpcsevents":"hcpcsevents.csv","microbiologyevents":"microbiologyevents.csv","poe":"poe.csv","prescriptions":"prescriptions.csv","services":"services.csv","diagnoses_icd":"diagnoses_icd.csv","d_icd_procedures":"d_icd_procedures.csv","drgcodes":"drgcodes.csv","emar_detail":"emar_detail.csv","labevents":"labevents.csv","pharmacy":"pharmacy.csv","poe_detail":"poe_detail.csv","procedures_icd":"procedures_icd.csv"}

In [ ]:
# Uncomment to unzip data
# from utils import unzip_files
# unzip_files(ed_path)
# unzip_files(mimic_iv_core)
# unzip_files(mimic_iv_icu)
# unzip_files(mimic_iv_hosp)

In [ ]:
df_edstays = pd.read_csv(ed_path+ed_filename_dict['edstays'])
df_triage = pd.read_csv(ed_path+ed_filename_dict['triage'])
df_vitalsign = pd.read_csv(ed_path+ed_filename_dict['vitalsign'])
df_medrecon = pd.read_csv(ed_path+ed_filename_dict['medrecon'])
df_pyxis = pd.read_csv(ed_path+ed_filename_dict['pyxis'])
df_diagnosis = pd.read_csv(ed_path+ed_filename_dict['diagnosis'])




In [ ]:
# vital sign contains multiple measurements, take the max, min and mean
gp = df_vitalsign.groupby(['stay_id'])
df_vitalsign_max = gp.max(numeric_only=True)#, min_count=1)
df_vitalsign_min = gp.min(numeric_only=True)#, min_count=1)
df_vitalsign_median = gp.median(numeric_only=True)

In [ ]:
df = pd.merge(df_edstays, df_triage, on = ['subject_id', 'stay_id'], how='left')
df = pd.merge(df, df_vitalsign_max, on = ['subject_id', 'stay_id'], how='left', suffixes=("_triage", "_vitalsign_max"))
df = pd.merge(df, df_vitalsign_min, on = ['subject_id', 'stay_id'], how='left', suffixes=("_vitalsign_max", "_vitalsign_min"))
df = pd.merge(df, df_vitalsign_median, on = ['subject_id', 'stay_id'], how='left', suffixes=("_vitalsign_min", "_vitalsign_median"))

# Each stay main contains multiple medrecon, pyxis and diagnosis
# df = pd.merge(df, df_vitalsign, on = ['subject_id', 'stay_id'], suffixes=("_triage", "_vitalsign"))
# df = pd.merge(df, df_medrecon, on = ['subject_id', 'stay_id'])
# df = pd.merge(df, df_pyxis, on = ['subject_id', 'stay_id'], suffixes=("_medrecon", "_pyxis"))
# df = pd.merge(df, df_diagnosis, on = ['subject_id', 'stay_id'])

# write to csv
#df.to_csv('ed_combined.csv', index=False)

In [ ]:
df['admitted'] = ~(df['hadm_id'].isnull())
df_icustays = pd.read_csv(mimic_iv_icu+icu_filename_dict["icustays"])
df['icu'] = df['hadm_id'].isin(df_icustays['hadm_id'])

In [ ]:
df_hosp = pd.read_csv(mimic_iv_core+core_filename_dict["admissions"])
df_death = df_hosp.loc[df_hosp['hadm_id'].isin(df['hadm_id']), ['hadm_id', 'deathtime']]
df = pd.merge(df, df_death, on = ['hadm_id'], how='left')
df['death'] = ~(df['deathtime'].isnull())


In [ ]:
# read saved csv
df = pd.read_csv('ed_combined.csv')

In [ ]:
df['label'] = 0
df.loc[df['death'], 'label']=3
df.loc[df['icu'] & (df['label']==0), 'label'] = 2
df.loc[df['admitted'] & (df['label']==0), 'label'] = 1


In [ ]:
import datetime
df_patients = pd.read_csv(mimic_iv_core+core_filename_dict["patients"])
df_patients['dod'] = pd.to_datetime(df_patients['dod'])
df = df.merge(df_patients[['subject_id', 'dod']], on='subject_id', how='left')
df['outtime'] = pd.to_datetime(df['outtime'])
df['deathtime_distance'] = df['dod'] - df['outtime']
df['death30'] = df['deathtime_distance'] < datetime.timedelta(days=30)

In [ ]:
df.to_csv('ed_combined.csv', index=False)

In [ ]:
# convert to numpy array for model
arr_stayid = df['stay_id'].to_numpy()
label = df['label'].to_numpy()
df.drop(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'acuity', 'chiefcomplaint', 'admitted', 'icu', 'deathtime', 'death', 'label'], axis=1)
data = df.to_numpy()
print(arr_stayid.shape)
print(label.shape)
print(data.shape)
import numpy as np
np.savez('vitalsign_feature.npz', id = arr_stayid, x = data, y=label)